<a href="https://colab.research.google.com/github/muhammed-saeed/AESLC/blob/master/phonological_similarity_PWLD_LSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
import pandas as pd
import numpy as np
from scipy.spatial import distance
import scipy.stats as stats
import csv

In [ ]:
from collections import defaultdict, Counter

In [ ]:
!wget https://github.com/uds-lsv/AWEs_phon_sim/raw/main/PWLD/data/phoneme_vectors/feature_dict.p
!wget https://raw.githubusercontent.com/uds-lsv/AWEs_phon_sim/main/PWLD/data/words_ipa/de.tsv

--2023-04-10 17:40:27--  https://github.com/uds-lsv/AWEs_phon_sim/raw/main/PWLD/data/phoneme_vectors/feature_dict.p
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/uds-lsv/AWEs_phon_sim/main/PWLD/data/phoneme_vectors/feature_dict.p [following]
--2023-04-10 17:40:27--  https://raw.githubusercontent.com/uds-lsv/AWEs_phon_sim/main/PWLD/data/phoneme_vectors/feature_dict.p
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 475975 (465K) [application/octet-stream]
Saving to: ‘feature_dict.p.1’

feature_dict.p.1    100%[===================>] 464.82K  --.-KB/s    in 0.01s   

2023-04-10 17:40:27 (35.3 MB/s

In [ ]:
# instantiate a dict phone --> feature vector
with open('/content/feature_dict.p', "rb") as input_file:
    phone2vec = pickle.load(input_file)

In [ ]:

word2ipa = {}

with open('/content/de.tsv') as csv_file:
    
    csv_reader = csv.DictReader(csv_file, delimiter='\t')#(csv_file, delimiter='\t', )
    
    
    #print(csv_reader)
    
    for row in csv_reader:
        word2ipa[row["Word"]] =  row["IPA transcription"]
        

In [ ]:
print(len(phone2vec))
print(phone2vec.keys())

3183
dict_keys(['a', 'aː', 'æ', 'æː', 'e', 'eː', 'ɤ', 'ɤː', 'h', 'i', 'iː', 'j', 'k', 'kʰ', 'kˀ', 'l', 'm', 'n', 'ŋ', 'o', 'ø', 'oː', 'p', 'pʰ', 'pˀ', 's', 'sˀ', 't', 'tʰ', 't̠ʃ', 't̠ʃʰ', 't̠ʃˀ', 'tˀ', 'u', 'uː', 'ɯ', 'ɯː', 'w', 'y', 'ʔ', 'b', 'd', 'dʲ', 'ɨ', 'ɨː', 'ʝ', 'k̟', 'lˠ', 'lʲ', 'nʲ', 'ɔ', 'ɔː', 'ɵ̞', 'ɵ̞ː', 'q', 'sʲ', 'tʲ', 'ʊ', 'æˤ', 'e̞ˤ', 'ɡ', 'ɢ', 'ɢʷ', 'ħ', 'k͈', 'k͈ʷ', 'kʷʰ', 'kʷʼ', 'kʼ', 'ø̞ˤ', 'p͈', 'pʼ', 'q͈', 'qʰ', 'q͈ʷ', 'qʷʰ', 'qʷʼ', 'qʼ', 'r', 's͈', 'ʃ', 'ʃ͈', 'ʃʷ', 'ʃ͈ʷ', 't͈', 't͈s', 'tsʰ', 't͈sʷ', 'tsʷʰ', 'tsʷʼ', 'tsʼ', 't̠͈ʃ', 't̠͈ʃʷ', 't̠ʃʷʰ', 't̠ʃʷʼ', 't̠ʃʼ', 'tʼ', 'x', 'x͈', 'xʷ', 'x͈ʷ', 'z', 'ʒ', 'χ', 'χ͈', 'χʷ', 'χ͈ʷ', 'a̟ː', 'ç̟', 'd̻', 'd̻z̻', 'e̞ː', 'ɜ', 'f', 'fʼ', 'ɡ̟ʲ', 'ɡʷ', 'ɣ̟ʲ', 'ɦ', 'ʝ̟', 'k̟ʲʰ', 'k̟ʲʼ', 'ɬʲ', 'ɬʲʼ', 'ɮʲ', 'n̻', 'o̞ː', 'qχ', 'qχʷ', 'qχʷʼ', 'qχʼ', 'ʁ', 'ʁʷ', 's̻', 'ʃʼ', 't̻ʰ', 't̻s̻', 't̻s̻ʼ', 't̻ʼ', 'v', 'x̟ʲ', 'z̻', 'ʔʷ', 'ʕ', 'a̟', 'e̞', 'ə̆', 'ɪ', 'o̞', 'ts', 'β', '˥˦', '˧', '˩˨', 'ãː', 'd̠ʒ', 'ɖ', 'ɖʐ', 'ʂ', 'ʈ', 'ʈʰ', 'ʈʂ'

In [ ]:
phone2vecKeys = list(phone2vec.keys())

In [ ]:
print(phone2vecKeys)

['a', 'aː', 'æ', 'æː', 'e', 'eː', 'ɤ', 'ɤː', 'h', 'i', 'iː', 'j', 'k', 'kʰ', 'kˀ', 'l', 'm', 'n', 'ŋ', 'o', 'ø', 'oː', 'p', 'pʰ', 'pˀ', 's', 'sˀ', 't', 'tʰ', 't̠ʃ', 't̠ʃʰ', 't̠ʃˀ', 'tˀ', 'u', 'uː', 'ɯ', 'ɯː', 'w', 'y', 'ʔ', 'b', 'd', 'dʲ', 'ɨ', 'ɨː', 'ʝ', 'k̟', 'lˠ', 'lʲ', 'nʲ', 'ɔ', 'ɔː', 'ɵ̞', 'ɵ̞ː', 'q', 'sʲ', 'tʲ', 'ʊ', 'æˤ', 'e̞ˤ', 'ɡ', 'ɢ', 'ɢʷ', 'ħ', 'k͈', 'k͈ʷ', 'kʷʰ', 'kʷʼ', 'kʼ', 'ø̞ˤ', 'p͈', 'pʼ', 'q͈', 'qʰ', 'q͈ʷ', 'qʷʰ', 'qʷʼ', 'qʼ', 'r', 's͈', 'ʃ', 'ʃ͈', 'ʃʷ', 'ʃ͈ʷ', 't͈', 't͈s', 'tsʰ', 't͈sʷ', 'tsʷʰ', 'tsʷʼ', 'tsʼ', 't̠͈ʃ', 't̠͈ʃʷ', 't̠ʃʷʰ', 't̠ʃʷʼ', 't̠ʃʼ', 'tʼ', 'x', 'x͈', 'xʷ', 'x͈ʷ', 'z', 'ʒ', 'χ', 'χ͈', 'χʷ', 'χ͈ʷ', 'a̟ː', 'ç̟', 'd̻', 'd̻z̻', 'e̞ː', 'ɜ', 'f', 'fʼ', 'ɡ̟ʲ', 'ɡʷ', 'ɣ̟ʲ', 'ɦ', 'ʝ̟', 'k̟ʲʰ', 'k̟ʲʼ', 'ɬʲ', 'ɬʲʼ', 'ɮʲ', 'n̻', 'o̞ː', 'qχ', 'qχʷ', 'qχʷʼ', 'qχʼ', 'ʁ', 'ʁʷ', 's̻', 'ʃʼ', 't̻ʰ', 't̻s̻', 't̻s̻ʼ', 't̻ʼ', 'v', 'x̟ʲ', 'z̻', 'ʔʷ', 'ʕ', 'a̟', 'e̞', 'ə̆', 'ɪ', 'o̞', 'ts', 'β', '˥˦', '˧', '˩˨', 'ãː', 'd̠ʒ', 'ɖ', 'ɖʐ', 'ʂ', 'ʈ', 'ʈʰ', 'ʈʂ', 'ʈʂʰ', 'ũː', 

In [ ]:
"ɡ" in englishPhoneSet

True

In [ ]:
englishPhoneSet = ['aɪ', 'aʊ', 'æ', 'b', 'ʧ', 'd', 'ð', 'ə', 'ɚ', 'ɛ', 'ɝ', 'f', 'g','ɡ', 'h', 'ʤ', 'i', 'ɪ', 'ʒ', 'k', 'l', 'm', 'n', 'ŋ', 'ɔɪ', 'oʊ', 'ɔ', 'p', 'r', 's', 'ʃ', 't', 'θ', 'u', 'ʊ', 'v', 'w', 'j', 'z', 'ʔ']

In [ ]:
intersection = list(set(phone2vecKeys).intersection(englishPhoneSet))
print(f"{len(intersection)} and {len(englishPhoneSet)}")

list(set(englishPhoneSet)-set(intersection) )

36 and 40


['g', 'ɝ', 'ʧ', 'ʤ']

In [ ]:
phone2vec

{'a': [1,
  0,
  -1,
  1,
  -1,
  -1,
  -1,
  1,
  1,
  0,
  1,
  -1,
  -1,
  -1,
  -1,
  -1,
  0,
  0,
  -1,
  0,
  0,
  0,
  1,
  -1,
  1,
  -1,
  -1,
  0,
  -1,
  -1,
  1,
  -1,
  -1,
  -1,
  0,
  -1,
  -1,
  0],
 'aː': [1,
  0,
  -1,
  1,
  -1,
  1,
  -1,
  1,
  1,
  0,
  1,
  -1,
  -1,
  -1,
  -1,
  -1,
  0,
  0,
  -1,
  0,
  0,
  0,
  1,
  -1,
  1,
  -1,
  -1,
  0,
  -1,
  -1,
  1,
  -1,
  -1,
  -1,
  0,
  -1,
  -1,
  0],
 'æ': [1,
  0,
  -1,
  1,
  -1,
  -1,
  -1,
  1,
  1,
  0,
  1,
  -1,
  -1,
  -1,
  -1,
  -1,
  0,
  0,
  -1,
  0,
  0,
  0,
  1,
  -1,
  1,
  1,
  -1,
  0,
  -1,
  -1,
  1,
  -1,
  -1,
  -1,
  0,
  -1,
  -1,
  0],
 'æː': [1,
  0,
  -1,
  1,
  -1,
  1,
  -1,
  1,
  1,
  0,
  1,
  -1,
  -1,
  -1,
  -1,
  -1,
  0,
  0,
  -1,
  0,
  0,
  0,
  1,
  -1,
  1,
  1,
  -1,
  0,
  -1,
  -1,
  1,
  -1,
  -1,
  -1,
  0,
  -1,
  -1,
  0],
 'e': [1,
  0,
  -1,
  1,
  -1,
  -1,
  -1,
  1,
  1,
  0,
  1,
  -1,
  -1,
  -1,
  -1,
  -1,
  0,
  0,
  -1,
  0,
  0,
  0,
  1,
  -1,
  

In [ ]:

word2ipa['Lösung']

'l øː z ʊ ŋ'

In [ ]:
# phoneme set
PHONEME_SET = set([ipa  for w, ipa in set(word2ipa.items()) for ipa in ipa.split() ])

In [ ]:
print(PHONEME_SET)
#{'j', 'ʃ', 'v', 'ŋ', 'l', 'øː', 'ʒ', 'oː', 'p', 't', 'f', 'eː', 'h', 'r', 'ɐ', 'a', 'm', 'ç', 'n', 'uː', 'ʏ', 'k', 'œ', 'yː', 'x', 'z', 'ɛ', 'ɡ', 'ʀ', 'iː', 'ɪ', 'ɛː', 'b', 's', 'd', 'ə', 'i', 'ʊ', 'ɔ', 'aː'}


{'j', 'ʃ', 'v', 'ŋ', 'l', 'øː', 'ʒ', 'oː', 'p', 't', 'f', 'eː', 'h', 'r', 'ɐ', 'a', 'm', 'ç', 'n', 'uː', 'ʏ', 'k', 'œ', 'yː', 'x', 'z', 'ɛ', 'ɡ', 'ʀ', 'iː', 'ɪ', 'ɛː', 'b', 's', 'd', 'ə', 'i', 'ʊ', 'ɔ', 'aː'}


In [ ]:
phone2dist = defaultdict(lambda: defaultdict(float))

for p1 in PHONEME_SET:
    for p2 in PHONEME_SET:
        
        dist = distance.hamming(phone2vec[p1], phone2vec[p2])#**(1/2)

        phone2dist[p1][p2] = dist

In [ ]:
phone2dist = defaultdict(lambda: defaultdict(float))

for p1 in list(phone2vec.keys()):
    for p2 in list(phone2vec.keys()):
        
        dist = distance.hamming(phone2vec[p1], phone2vec[p2])#**(1/2)

        phone2dist[p1][p2] = dist

In [ ]:
Enphone2dist = defaultdict(lambda: defaultdict(float))

for p1 in englishPhoneSet:
    for p2 in englishPhoneSet:
        
        dist = distance.hamming(phone2vec[p1], phone2vec[p2])#**(1/2)

        phone2dist[p1][p2] = dist

KeyError: ignored

In [ ]:


phone2dist['z']['s'], phone2dist['p']['b'], phone2dist['p']['z'], phone2dist['p']['ʊ'],  

(0.02631578947368421, 0.02631578947368421, 0.2631578947368421, 0.5)

In [ ]:
# String-based Levenstein Distance (PWLD)
def LD(s, t):
        ''' From Wikipedia article; Iterative with two matrix rows. '''
        if s == t: return 0
        elif len(s) == 0: return len(t)
        elif len(t) == 0: return len(s)
        v0 = [None] * (len(t) + 1)
        v1 = [None] * (len(t) + 1)
        for i in range(len(v0)):
            v0[i] = i
        for i in range(len(s)):
            v1[0] = i + 1
            for j in range(len(t)):
                cost = 0 if s[i] == t[j] else 1
                v1[j + 1] = min(v1[j] + 1, v0[j + 1] + 1, v0[j] + cost)
            for j in range(len(v0)):
                v0[j] = v1[j]
                
        return v1[len(t)]

In [ ]:


# Phonologically-Weighted Levenstein Distance (PWLD)

def PWLD(s, t):
        ''' From Wikipedia article; Iterative with two matrix rows. '''
        if s == t: return 0
        elif len(s) == 0: return len(t)
        elif len(t) == 0: return len(s)
        v0 = [None] * (len(t) + 1)
        v1 = [None] * (len(t) + 1)
        for i in range(len(v0)):
            v0[i] = i
        for i in range(len(s)):
            v1[0] = i + 0.5
            for j in range(len(t)):
                cost = 0 if s[i] == t[j] else phone2dist[s[i]][t[j]]
                v1[j + 1] = min(v1[j] + 0.5, v0[j + 1] + 0.5, v0[j] + cost)
            for j in range(len(v0)):
                
                
                v0[j] = v1[j]
                
        return v1[len(t)]

In [ ]:
!pip install phonemizer
!pip install pyespeak

!pip install speake3  # Python 3
!pip install espeakng

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement pyespeak (from versions: none)
ERROR: No matching distribution found for pyespeak
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!apt-get install -y espeak


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 24 not upgraded.
Need to get 1,220 kB of archives.
After this operation, 3,060 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libportaudio2 amd64 19.6.0-1build1 [65.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libsonic0 amd64 0.2.0-8 [13.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 espeak-data amd64 1.48.04+dfsg-8build1 [932 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/universe amd64 libespeak1 amd64 1.48.04+dfsg-8build1 [147 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal/universe amd64 espeak amd64 1.48.04+dfsg-8build1 [61.8 kB]
Fetched 1,220 kB i

In [ ]:
from phonemizer import phonemize
from phonemizer.backend import EspeakBackend

backend = EspeakBackend('en-us')

# Get phonemes from word
def get_phonemes(text):
    return phonemize(text, language='en-us', backend='espeak')

word1 = get_phonemes("hello")
print(word1)
word2 = get_phonemes("hi")
print(word2)

həloʊ 
haɪ 


In [ ]:
LD('k ɪ ʀ ç ə'.split(), 'k ɪ ʀ ʃ ə'.split())

1

In [ ]:
PWLD(list(word1.strip()), list(word1.strip()))

0

In [ ]:
list(word1.strip())

['h', 'ə', 'l', 'o', 'ʊ']

In [ ]:
PWLD(word1.split(), word1.split())

0

In [ ]:
PWLD(word2ipa['Lösung'].split(), word2ipa['Lösung'].split())

0

In [ ]:
PWLD(word2ipa['Lösung'].split(), word2ipa['schließen'].split())

1.2105263157894737

In [ ]:
PWLD(word2ipa['schlossen'].split(), word2ipa['schließen'].split())

0.21052631578947367